# CONVERT FILES WITH DUCK_DB

In [1]:
import pandas as pd
import duckdb
import os
import time
import numpy as np
import sys
sys.path.insert(0, os.path.abspath(".."))
from utils import time, data_processor, constants

In [2]:
OUTPUT_PATH = r"..\data_processed\\guo_subs_europee\\subs_eu\\" 

eu27_countries = constants.eu
all_countries = constants.all_countries

# drop 'CN' from the list
all_countries.remove('CN')

## Ownership history

In [3]:
TEMP_TABLE_FIRMOGRAPHICS = "..\\data_processed\\firmographics_processed\\firmographics_full\\" 
TEMP_TABLE_KEY_FINANCIALS = "..\\data_processed\\key_financials_detailed_processed\\"

def get_ownership_data(year, country, path=None):
    print(f"{country} - {year}...")
    COUNTRY_KEY_FINANCIALS = TEMP_TABLE_KEY_FINANCIALS + f"key_financials_detailed_{country}.parquet"
    COUNTRY_FIRMOGRAPHICS = TEMP_TABLE_FIRMOGRAPHICS + f"{country}.parquet"

    query = f"""
        SELECT
            main.subsidiary_bvd_id,
            main.guo_25,
            
            firmographics_sub.nuts2 AS subsidiary_nuts2,
            firmographics_sub.nace_rev_2_core_code_4_digits_ AS subsidiary_nace4,
            firmographics_sub.city_native_ AS city,
            firmographics_sub.region_in_country AS subsidiary_region_in_country,
            firmographics_guo.nuts2 AS guo_nuts2,
            firmographics_guo.nace_rev_2_core_code_4_digits_ AS guo_nace4,
            firmographics_guo.type_of_entity AS guo_type_of_entity,
            firmographics_guo.status AS guo_status,
            firmographics_guo.region_in_country AS guo_region_in_country,

            key_financials_detailed.operating_revenue_turnover_,
            key_financials_detailed.number_of_employees,
        FROM 
            '{path}' AS main
        LEFT JOIN 
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_sub
        ON 
            main.subsidiary_bvd_id = firmographics_sub.bvd_id_number
        LEFT JOIN
            '{COUNTRY_FIRMOGRAPHICS}' AS firmographics_guo
        ON
            main.guo_25 = firmographics_guo.bvd_id_number
        LEFT JOIN 
            (SELECT * 
             FROM '{COUNTRY_KEY_FINANCIALS}'
             WHERE 
                year = {year}
            ) AS key_financials_detailed
        ON
            main.guo_25 = key_financials_detailed.bvd_id_number   -- CHANGE HERE FOR SUBS
        AND
            key_financials_detailed.year = {year}

        WHERE 
            main."type_of_relation" = 'GUO 25'
        AND 
            main."guo_25" LIKE '{country}%'
    """
    

    conn = duckdb.connect()
    df = conn.execute(query).fetchdf()
    return df

In [4]:
@time.time_it
def fetch_and_convert_to_excel(year, country, path, output_path):
    df = get_ownership_data(year, country, path)
    os.makedirs(output_path, exist_ok=True)
    slices = data_processor.split_in_chunks(df, 5)
    
    for i, s in enumerate(slices):
        s.to_csv(f"{output_path}\\{country}_{year}_chunk_{i+1}.csv", index=False)
        print(f"Data for {country} - {year} - slice_{i+1} has been converted.")
        

### Test

In [5]:
# path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"
country = "AT"
year = "2007"
path = "..\\data_raw\\ownership_history\\links_2007\\*.parquet"

df = get_ownership_data(
    path=path,
    year=year,
    country=country,
)
# fetch_and_convert_to_excel(country=country, year=year, path=path, output_path=output_path)

AT - 2007...


## Query data

In [6]:
import time
import json

errors = {}

total_start = time.time()
start_time = time.time()

for country in all_countries:
# for country in ['AT']:
    for year in range(2007, 2024):
        year = str(year)
        try:
            path = f"..\\data_raw\\ownership_history\\links_{year}\\*.parquet"
            fetch_and_convert_to_excel(year, country, path, OUTPUT_PATH)  
            end_time = time.time()
            duration = end_time - start_time
            print(f"{country} - {year} converted in {duration:.2f} seconds.")
        except Exception as e:
            errors[f"{country} - {year}"] = str(e)
            print(f"Error {country} - {year}")
            print(e)
            continue


end_time = time.time()
duration = end_time - start_time
total_end = time.time()
total_duration = total_end - total_start
print(f"Total execution time: {total_duration:.2f} seconds.")

AF - 2007...
Data for AF - 2007 - slice_1 has been converted.
Data for AF - 2007 - slice_2 has been converted.
Data for AF - 2007 - slice_3 has been converted.
Data for AF - 2007 - slice_4 has been converted.
Data for AF - 2007 - slice_5 has been converted.
Execution time: 5.3152 seconds
AF - 2007 converted in 5.32 seconds.
AF - 2008...
Data for AF - 2008 - slice_1 has been converted.
Data for AF - 2008 - slice_2 has been converted.
Data for AF - 2008 - slice_3 has been converted.
Data for AF - 2008 - slice_4 has been converted.
Data for AF - 2008 - slice_5 has been converted.
Execution time: 7.4500 seconds
AF - 2008 converted in 12.77 seconds.
AF - 2009...
Data for AF - 2009 - slice_1 has been converted.
Data for AF - 2009 - slice_2 has been converted.
Data for AF - 2009 - slice_3 has been converted.
Data for AF - 2009 - slice_4 has been converted.
Data for AF - 2009 - slice_5 has been converted.
Execution time: 8.2232 seconds
AF - 2009 converted in 20.99 seconds.
AF - 2010...
Data fo

: 

: 

In [ ]:
import json 

with open("errors/errors_guos.json", "r") as f:
    errors = json.load(f)

errors

{}

## Zip the files

In [ ]:
ZIP_OUTPOUT = "..\\zipped_files\\guos_eu"

data_processor.compress_csv_files(folder_to_zip=OUTPUT_PATH, zip_output=ZIP_OUTPOUT)